In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt


/Users/rongfan/Documents/GitHub/SmartTrader/venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [63]:

# Load the trained model
model_path = 'best_lstm_multitarget3.keras'  # Path to your saved model
model = load_model(model_path)
print("Model loaded successfully.")

# Load the original data for scaling and context
data = pd.read_csv("../data/NVDA.csv", skiprows=2, index_col=0)
# Convert the index to a datetime type for better handling
data.index = pd.to_datetime(data.index)
data.columns = ['Adj Close','Close','High','Low','Open','Volume']
print(data.head())

print("Data loaded successfully.")


Model loaded successfully.
            Adj Close    Close     High      Low     Open     Volume
Date                                                                
2019-01-02   3.378090  3.40550  3.46200  3.25125  3.26600  508752000
2019-01-03   3.173996  3.19975  3.37900  3.19225  3.34475  705552000
2019-01-04   3.377346  3.40475  3.44325  3.24250  3.27350  585620000
2019-01-07   3.556145  3.58500  3.62225  3.41075  3.46250  709160000
2019-01-08   3.467614  3.49575  3.66950  3.42250  3.66725  786016000
Data loaded successfully.


In [65]:
from sklearn.preprocessing import MinMaxScaler

# Extract numeric values
features = data[['Open', 'High', 'Low', 'Close']].values

# Normalize the features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_features = scaler.fit_transform(features)

print(f"Scaled features shape: {scaled_features.shape}")


Scaled features shape: (1489, 4)


In [66]:
look_back = 60
# Check if enough historical data is available
if len(data) < look_back:
    raise ValueError("Not enough historical data to make predictions.")


In [67]:
# Use the last `look_back` days from the dataset
input_data = scaled_features[-look_back:]
input_data = input_data.reshape(1, look_back, input_data.shape[1])  # Reshape to match model input


In [69]:
def predict_multiple_days(model, scaler, recent_data, target_date, data_index, business_days=5):
    """
    Predict prices for a specific day and subsequent business days.
    :param model: Trained LSTM model
    :param scaler: MinMaxScaler used for normalization
    :param recent_data: Most recent `look_back` days of scaled data with 4 features
    :param target_date: The future date to predict (YYYY-MM-DD)
    :param data_index: The index of dates in the dataset
    :param business_days: Number of business days to predict after the target date
    :return: DataFrame with predicted Open, High, Low, Close prices for the specified period
    """
    from datetime import datetime, timedelta
    import pandas as pd

    # Ensure recent_data has the correct shape
    if recent_data.shape[1] != 4:
        raise ValueError(f"Expected 4 features (Open, High, Low, Close), but got {recent_data.shape[1]} features.")

    # Initialize input and predictions
    current_input = recent_data.reshape(1, recent_data.shape[0], recent_data.shape[1])  # (1, look_back, 4)
    predictions = []
    dates = []

    # Start with the target date
    predicted_date = datetime.strptime(target_date, "%Y-%m-%d")

    # Predict for the target date and subsequent business days
    for step in range(business_days):
        print(f"Predicting day {step + 1}")
        prediction = model.predict(current_input)

        # Denormalize the prediction
        denormalized_prediction = scaler.inverse_transform(prediction)
        predictions.append(denormalized_prediction[0])
        dates.append(predicted_date)

        # Update the sliding window
        new_input_row = np.vstack([current_input[0, 1:], prediction])  # Slide the input window
        current_input = new_input_row.reshape(1, current_input.shape[1], current_input.shape[2])

        # Move to the next business day
        predicted_date += timedelta(days=1)
        while predicted_date.weekday() >= 5:  # Skip weekends (5=Saturday, 6=Sunday)
            predicted_date += timedelta(days=1)

    # Create DataFrame for predictions
    df_predictions = pd.DataFrame(predictions, columns=['Open', 'High', 'Low', 'Close'])
    df_predictions['Date'] = dates
    df_predictions.set_index('Date', inplace=True)

    return df_predictions


In [71]:
future_date = "2024-12-27"  # Starting from this date

# Ensure recent_data includes the last 60 days with 4 features
recent_data = scaled_features[-look_back:, :4]

# Predict for the specific day and the next 4 business days
predicted_df = predict_multiple_days(
    model=model,
    scaler=scaler,
    recent_data=recent_data,
    target_date=future_date,
    data_index=data.index,
    business_days=5
)

# Display the predictions
print(predicted_df)


Predicting day 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
Predicting day 2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
Predicting day 3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Predicting day 4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
Predicting day 5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
                  Open        High         Low       Close
Date                                                      
2024-12-27  131.705338  131.267105  124.192284  127.582771
2024-12-30  130.009354  129.427811  122.657562  125.858253
2024-12-31  128.228287  127.452454  120.999397  124.004692
2025-01-01  126.422401  125.458443  119.307053  122.152451
2025-01-02  124.567139  123.512878  117.583298  120.200470
